<a href="https://colab.research.google.com/github/raut-sam/Lets-try/blob/master/WIMLDS_31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch

     |████████████████████████████████| 748.9MB 18kB/s 


In [0]:
import torch
from torch import nn
import numpy as np


In [0]:
#PREPROCESSING
text=['hey how are you','iam fine','good iam great','we can meet','great sure']   #dataset 
chars=set(''.join(text))
int2char=dict(enumerate(chars))
char2int={char:ind for ind, char in int2char.items()}

In [6]:
print(int2char)

{0: ' ', 1: 'i', 2: 'u', 3: 'o', 4: 'g', 5: 'r', 6: 'f', 7: 'd', 8: 's', 9: 'e', 10: 'c', 11: 'w', 12: 'h', 13: 'm', 14: 'y', 15: 'n', 16: 'a', 17: 't'}


In [7]:
print(char2int)

{' ': 0, 'i': 1, 'u': 2, 'o': 3, 'g': 4, 'r': 5, 'f': 6, 'd': 7, 's': 8, 'e': 9, 'c': 10, 'w': 11, 'h': 12, 'm': 13, 'y': 14, 'n': 15, 'a': 16, 't': 17}


In [0]:
#Finding the length of the 
maxlen=len(max(text,key=len))
#Padding
#A simple loop that loops through the list of the sentences and adds a ' ' whitespace until the length of the sentence matches the length of the longest sentence
for i in range (len(text)):
  while len(text[i])<maxlen:
    text[i]+=' '

In [33]:
#Creating lists that will hold our input and target sequences
input_seq=[]
target_seq=[]
for i in range(len(text)):
  input_seq.append(text[i][: -1])
  target_seq.append(text[i][1 :])
  print("Input sequence:{} \n Target sequence:{}".format(input_seq[i],target_seq[i]))

Input sequence:hey how are yo 
 Target sequence:ey how are you
Input sequence:iam fine       
 Target sequence:am fine       
Input sequence:good iam great 
 Target sequence:ood iam great 
Input sequence:we can meet    
 Target sequence:e can meet    
Input sequence:great sure     
 Target sequence:reat sure     


In [0]:
for i in range(len(text)):
  input_seq[i]=[char2int[character] for character in input_seq[i]]
  target_seq[i]=[char2int[character]for character in target_seq[i]]

In [0]:
dict_size=len(char2int)
seq_len=maxlen-1
batch_size=len(text)

In [0]:
def one_hot_encode(sequence,dict_size,seq_len,batch_size):
  features=np.zeros((batch_size,seq_len,dict_size),dtype=np.float32)
  for i  in range(batch_size):
    for u in range(seq_len):
      features[i,u,sequence[i][u]]=1
  return features

In [0]:
input_seq=one_hot_encode(input_seq,dict_size,seq_len,batch_size)

In [0]:
input_seq=torch.from_numpy(input_seq)
target_seq=torch.Tensor(target_seq) 

In [42]:
is_cuda= torch.cuda.is_available()
if is_cuda:
  device=torch.device("cuda")
  print("GPU is available")
else:
  device=torch.device("cpu")
  print("GPU is not available, CPU is used")

GPU is not available, CPU is used


In [0]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters #self is an object that we are defining 
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer(i.e every node inone layer is taking input from previous layer) not every layer is FC
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [44]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

print (model)
# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Model(
  (rnn): RNN(18, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=18, bias=True)
)


In [45]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.3708
Epoch: 20/100............. Loss: 1.9875
Epoch: 30/100............. Loss: 1.6290
Epoch: 40/100............. Loss: 1.2928
Epoch: 50/100............. Loss: 1.0021
Epoch: 60/100............. Loss: 0.7729
Epoch: 70/100............. Loss: 0.5949
Epoch: 80/100............. Loss: 0.4529
Epoch: 90/100............. Loss: 0.3456
Epoch: 100/100............. Loss: 0.2694


In [0]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [0]:
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)